In [87]:
pip install requests beautifulsoup4


Scraping des données et stockage dans excel :  Script de scraping permetant d’extraire les données DONE
• Données extraites en Excel DONE

In [89]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL de la page à scraper
url = "http://www.tunisie-annonce.com/AnnoncesImmobilier.asp"

# Envoyer une requête GET à la page
response = requests.get(url)
response.encoding = "ISO-8859-1"  # S'assurer du bon encodage

# Vérifier que la requête a réussi
if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")

    # Trouver toutes les lignes d'annonces (les <tr> qui contiennent les annonces)
    annonces = soup.find_all("tr", class_="Tableau1")

    # Liste pour stocker les données extraites
    resultats = []

    for annonce in annonces:
        cols = annonce.find_all("td")

        if len(cols) >= 6:  # Vérifier qu'il y a assez de colonnes
            region = cols[1].get_text(strip=True)
            nature = cols[3].get_text(strip=True)
            type_annonce = cols[5].get_text(strip=True)
            texte_annonce = cols[7].get_text(strip=True)
            prix = cols[9].get_text(strip=True)
            date_modif = cols[11].get_text(strip=True)

            # Ajouter à la liste des résultats
            resultats.append({
                "Région": region,
                "Nature": nature,
                "Type": type_annonce,
                "Texte": texte_annonce,
                "Prix": prix,
                "Modifiée": date_modif
            })

    # Convertir les données en DataFrame Pandas
    df = pd.DataFrame(resultats)

    # Exporter vers un fichier Excel
    df.to_excel("annonces.xlsx", index=False, engine='openpyxl')

    print("Les données ont été enregistrées dans annonces.xlsx")

else:
    print("Erreur lors du chargement de la page")


Les données ont été enregistrées dans annonces.xlsx


Installation des dépendances

In [101]:
!pip install fastapi uvicorn nest_asyncio pyngrok requests pandas openpyxl bs4


Implémentation de l’API avec FastAPI

In [104]:
!ngrok config add-authtoken 2ub8U4p67DYslVunez1y39LwG3G_679fw6hYuxbWNQZy3QTbW

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


API REST opérationnelle permetant d’extraire et d’accéder aux données sur POSTMAN.  DONE

In [108]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from fastapi import FastAPI
import nest_asyncio
from pyngrok import ngrok
import uvicorn

# Initialiser FastAPI
app = FastAPI()

# Stockage temporaire des annonces
annonces_data = []

# Fonction pour scraper les annonces
def scraper():
    global annonces_data
    url = "http://www.tunisie-annonce.com/AnnoncesImmobilier.asp"
    response = requests.get(url)
    response.encoding = "ISO-8859-1"  # S'assurer du bon encodage

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        annonces = soup.find_all("tr", class_="Tableau1")

        resultats = []
        for annonce in annonces:
            cols = annonce.find_all("td")
            if len(cols) >= 6:  # Vérifier qu'il y a assez de colonnes
                resultats.append({
                    "Région": cols[1].get_text(strip=True),
                    "Nature": cols[3].get_text(strip=True),
                    "Type": cols[5].get_text(strip=True),
                    "Texte": cols[7].get_text(strip=True),
                    "Prix": cols[9].get_text(strip=True),
                    "Modifiée": cols[11].get_text(strip=True)
                })

        annonces_data = resultats
        return resultats
    return []

# Endpoint pour récupérer les annonces
@app.get("/annonces")
def get_annonces():
    return {"total": len(annonces_data), "data": annonces_data}

# Endpoint pour lancer un nouveau scraping
@app.post("/scrape")
def scrape():
    global annonces_data
    annonces_data = scraper()
    return {"message": "Scraping terminé et données mises à jour", "total": len(annonces_data)}

# Autoriser l'exécution asynchrone dans Google Colab
nest_asyncio.apply()

# Lancer le serveur FastAPI avec ngrok
port = 8000
ngrok_tunnel = ngrok.connect(port)
print(f"🚀 API accessible ici : {ngrok_tunnel.public_url}/docs")

# Démarrer l'application FastAPI
uvicorn.run(app, host="0.0.0.0", port=port)


🚀 API accessible ici : https://2f5d-35-222-104-157.ngrok-free.app/docs


INFO:     Started server process [234]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [234]


https://16d6-35-222-104-157.ngrok-free.app/docs